<img width="400" src="https://nomeroff.net.ua/images/logo/nomeroff_net.svg" alt="Nomeroff Net. Automatic numberplate recognition system" align="left"/>

## The example demonstrates license plate number detection.

In [1]:
#Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "" 
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [ ]:
import matplotlib.image as mpimg
import os
import pandas as pd

In [ ]:
# Import all necessary libraries.
import numpy as np
import sys
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
import copy

# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('../../../')
sys.path.append(NOMEROFF_NET_DIR)

In [ ]:
from NomeroffNet.YoloV5Detector import Detector

detector = Detector()
detector.load()

In [ ]:
from NomeroffNet.BBoxNpPoints import NpPointsCraft, getCvZoneRGB, convertCvZonesRGBtoBGR, reshapePoints

npPointsCraft = NpPointsCraft()
npPointsCraft.load()

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
rootDir = '/mnt/storage1/datasets/numberplate/eu_dataset_photos_test/'

photos = pd.read_csv('photos_eu_dataset.csv')
photos = photos.set_index(['photoId'])

In [ ]:
for dirName, subdirList, fileList in os.walk(rootDir):
    for fname in fileList:
        try:
            baseName = os.path.splitext(os.path.basename(fname))[0]
            img_path = os.path.join(dirName, fname)
            print(img_path)
            img = mpimg.imread(img_path)   

            targetBoxes = detector.detect_bbox(copy.deepcopy(img))
            targetBoxes = targetBoxes

            all_points = npPointsCraft.detect(img, targetBoxes)
            all_points = [ps for ps in all_points if len(ps)]

            # cut zones
            toShowZones = [getCvZoneRGB(img, reshapePoints(rect, 1)) for rect in all_points]
            zones = convertCvZonesRGBtoBGR(toShowZones)

            if len(toShowZones) > 1:
                areas = [zone.shape[0]*zone.shape[1] for zone in zones]
                zone = toShowZones[np.argmax(areas)]
            else:
                zone = toShowZones[0]

            text = photos.loc[int(baseName)]['npText']
            imgpath = "/mnt/storage1/datasets/numberplate/eu_numberplates_test/{}_{}.png".format(baseName, text)
            mpimg.imsave(imgpath, zone)

            # plt.axis("off")
            # plt.imshow(zone)
            # plt.show()
        except Exception as ex:
            print('[ERROR]', ex)